# Scraping the data

In [419]:
import pandas as pd
import numpy as np
import gzip
import datetime
import requests
import sys
import os

from StringIO import StringIO
from PIL import Image

# import pickle

In [55]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')


In [538]:

path  = 'data/price_history/reviews_Musical_Instruments_5.json.gz'

In [539]:
df = getDF(path)

##if slow
## first N lines
# N = 10
# PARSE = parse(path)
# d = next(PARSE)
# counter = 0
# while len(d) >0 and counter <7:
#     counter +=1;
#     print d['asin']
#     d = next(PARSE)


In [540]:
# Define items by ASIN
items = df.asin.value_counts()
item_names = list(items.index.get_values())
items.head()

B003VWJ2K8    163
B0002E1G5C    143
B0002F7K7Y    116
B003VWKPHC    114
B0002H0A3S     93
Name: asin, dtype: int64

In [541]:
# Save #
# check the path !!!
print path
# items.to_pickle(path[:-7]+'counts')
# items.to_csv(   path[:-7]+'counts.csv')

# print datetime.date.fromtimestamp(df.unixReviewTime.min())
# print datetime.date.fromtimestamp(df.unixReviewTime.max())

data/price_history/reviews_Musical_Instruments_5.json.gz


In [542]:
# Size of obeject and filenames

print  sys.getsizeof(df)/2**20, 'Mb'

filenames = [os.path.basename(i)[:-4] for i in glob('data/price_history/new/*.png',)]
print 'There are %d objects in the picture folder' % len(filenames)

8 Mb
There are 12101 objects in the picture folder


### Analyze ratings

In [545]:
path_r = r'data/price_history/ratings_Musical_Instruments.csv'

In [546]:
columns = ['User', 'ASIN', 'Rating', 'Timestamp']
ratings_df = pd.read_csv(path_r, header=None, names = columns) 
print sys.getsizeof(ratings_df)/2**20, 'Mb'
ratings_df.head()
# 

61 Mb


,User,ASIN,Rating,Timestamp
0,A1YS9MDZP93857,0006428320,3.0,1394496000
1,A3TS466QBAWB9D,0014072149,5.0,1370476800
2,A3BUDYITWUSIS7,0041291905,5.0,1381708800
3,A19K10Z0D2NTZK,0041913574,5.0,1285200000
4,A14X336IB4JD89,0201891859,1.0,1350432000


In [547]:
r = ratings_df.ASIN.value_counts()

In [558]:
# intersection if indexes such that 
#  reviews >4, ratings>50
idx = items[items>4].index.intersection(r[r>50].index)
len(items[idx])


517

# Scrap

In [553]:
def getImage(asin,  tag =  'new', time_sleep = 5):
    if len(asin)==10:

        image_name = 'data/price_history/%s/%s.png' % (tag,asin)
        if os.path.exists('data/price_history/'+tag):
            url = r'https://charts.camelcamelcamel.com/us/%s/%s.png?force=1&zero=1&w=1000&h=500&desired=false&legend=1&ilt=1&tp=all&fo=0&lang=en'  \
            % (asin, tag)

            if not os.path.isfile(image_name):
#                 print  '... created \t',image_name
                try:
                    r = requests.get(url)
                    time.sleep(time_sleep+2*np.random.random()-1)
                    if r.status_code == 200:
                        i = Image.open(StringIO(r.content))
                        i.save(image_name)
                    
                    else:
                        print 'status_code=', r.status_code
                except Exception as e:
                    print(e.message)

                
        else:
            print 'no such folder -->', tag
    else:
        print('Wrong ASIN provided')




In [564]:
len(list(idx))

517

In [565]:

counter = 0 ## for progress bar
item_list = list(idx)
for item in item_list:
    tag =  'amazon-new' #'new' 'amazon',
    getImage(item,tag,time_sleep=5)
    counter +=1
    sys.stdout.write('\r')
    sys.stdout.write("[%-20s] %s\t %d/%d" % ('='*(20*counter/len(item_list)), item, counter,len(item_list)))
    time.sleep(0.001)
    sys.stdout.flush()
    

[====================] B002Q0WT6U	 517/517

<div class="alert alert-block alert-info">
</div>


# Extra things

In [420]:
# ## thetracktor
# t = page.text
# r = re.compile('loadPrices\((\d+),')
# id = re.findall(r, t)

# json_url = r'https://thetracktor.com/ajax/prices/?id=%s&days=90' % id[0]
# print json_url
# # json_url = requests.get(json_url)
